# LINGUAGEM NATURAL E INTELIGÊNCIA ARTIFICIAL


## CLASSIFICAÇÃO AUTOMÁTICA DE TEXTO

Carlos Catalão Alves  
13 Maio, 2018

#### Aplicação:

Classificar textos de acções de divulgação científica, com aplicação de aprendizagem automática supervisionada.  
  
  

Os textos utilizados nesta aplicação são obtidos a partir de dois ficheiros em formato CSV:  

1. textos_treino.csv (com textos classificados, para treino do modelo)   
2. textos_classificar.csv (com os novos textos para classificação automática) 

O resultado final será a criação de dois ficheiros com os novos textos já classificados,   
com indicação da respectiva probabilidade:

1. textos_classificados.csv (ficheiro csv, delimitado com tabs)  
2. textos_classifcados.xlsx (ficheiro excel)



#### Livrarias python

In [2]:
import re
import pandas as pd
from collections import OrderedDict

#### Livrarias NLTK - Natural Langage Processing Toolkit

In [3]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

#### Livrarias SciKit Learn

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

#### Variáveis

In [5]:
corpus = []
novo_corpus = []
categorias = [ "outra","Astronomia","Biologia","Geologia","Engenharia","Patrimonio"]

TREINO = "textos_treino.csv"
CLASSIFICAR = "textos_classificar.csv"
RESULTADOS = "textos_classificados"

#### Funções

In [6]:
def tokenizar(texto, stemizar=True, lingua="portuguese"):

    # Usa o SnowballStemmer do NLTK para português
    stemizador = SnowballStemmer(lingua)

    # Tokeniza por frase e por palavra
    tokens = [word.lower() for frase in nltk.sent_tokenize(texto) for word in nltk.word_tokenize(frase)]
    
    # Aplica NLTK stopwords 
    stop_words_pt = set(stopwords.words(lingua)) 
    
    # Filtra as palavras que não têm letras, e as que estão na lista de stopwords
    tokens_filtrados = []
    for token in tokens:
        if re.search("[a-zA-Z]", token):
            if token not in stopwords.words(lingua):
                if token not in stop_words_pt:
                    tokens_filtrados.append(token)
                if stemizar:
                    stems = [stemizador.stem(token) for token in tokens_filtrados]
    return stems


#### Textos para treino

In [7]:
# Importando os dados de treino
dados_treino = pd.read_csv(TREINO, delimiter = "\t", quoting = 3)

#### Pré-processamento dos textos de treino

In [8]:
print("\nA treinar com textos classificados ...\n")
for i in range(len(dados_treino)):    
    termos = tokenizar(dados_treino["texto"][i], stemizar=True, lingua="portuguese")    
    # refaz cada linha com os tokens processados
    termos = ' '.join(termos)
    corpus.append(termos)


A treinar com textos classificados ...



#### Calcular a matriz TF-IDF

In [9]:
# Vectorizar só as 100 palavras mais frequentes
cv = CountVectorizer(max_features = 100)

# Obter a variável independente X (ie., os textos já classificados)
X = cv.fit_transform(corpus).toarray()
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X)

# Obter a variável independente y (a classificação dos textos)
# ":" todas as linhas, e "2" a coluna com a classific
y = dados_treino.iloc[:, 2].values 

#### Aplicar o modelo 

In [10]:
# Aplicar o modelo Multinominal Naive Bayes
clf = MultinomialNB().fit(X_tfidf, y)

#### Importar novos textos para classificação

In [11]:
dados_classificar = pd.read_csv(CLASSIFICAR, delimiter = "\t", quoting = 3)

#### Pré-processamento dos novos textos

In [12]:
print("A classificar novos textos ...\n")
for i in range(len(dados_classificar)):    
    termos = tokenizar(dados_classificar["texto"][i], stemizar=True, lingua="portuguese")    
    # refaz cada linha com os termos processados
    termos = ' '.join(termos)
    novo_corpus.append(termos)

A classificar novos textos ...



#### Calcular a matiz TF-IDF dos novos textos

In [13]:
X_novo = cv.transform(novo_corpus)
X_novo_tfidf = tfidf_transformer.transform(X_novo)

#### Previsão de classificações e respectivas probabilidades

In [14]:
previsoes = clf.predict(X_novo_tfidf)
probabilidades = clf.predict_proba(X_novo_tfidf)

nova_classificacao = []
for i in range(len(dados_classificar)): 
    p = previsoes[i]
    #pb = p - 1 # Obter a coluna com a probabilidade válida

    line = {
            "titulo":dados_classificar["titulo"][i],
            "texto":dados_classificar["texto"][i],
            "previsao":p,
            "categoria":categorias[previsoes[i]],
            #p - 1 para a coluna com a probabilidade válida
            "probabilidade":(probabilidades[i][p-1]) * 100,             
            }
    OrderedDict(line) # para preservar a ordem das colunas
    nova_classificacao.append(line)


#### Guardar os novos textos, já classificados

In [15]:
# Criar o dataframe com os textos classificados
df = pd.DataFrame(nova_classificacao)
# Manter a ordem das colunas no dataframe 
df = df[[ k for k in nova_classificacao[0].keys()]] 

# Salvar num ficheiro CSV
df.to_csv(RESULTADOS + ".csv", index=False, sep='\t')

# Salvar num ficheiro excel
df.to_excel(RESULTADOS + ".xlsx", index=False)

print("Resultados gravados em excel e csv.\n")

Resultados gravados em excel e csv.

